In [105]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import math 

# Global variables
print("Configuration")
path_root = "G:\\CIAT\\Code\\CWR\\ideal_diet\\src\\"
path_inputs = path_root +"inputs\\"
path_outputs = path_root + "outputs\\"

print("\tSetting parameters")
fao_element = 664

print("\tLoading parameters")
groups = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='groups')
diets = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='diets')
years = pd.read_excel(path_inputs + "configurations.xlsx", sheet_name='years')

print("\tOther things")
years_fao_colnames = ["Y"+str(y.year) for y in years.itertuples(index=True, name='Pandas')]

print("\tSuccessful")

Configuration
	Setting parameters
	Loading parameters
	Other things
	Successful


In [111]:
print("Merging fao data with groups")

print("\tLoading fao data")
data_fao = pd.read_csv(path_inputs + "faostat-food_supply.csv", encoding = "ISO-8859-1")

print("\tMerging fao data with groups")
data_fao_groups = pd.merge(data_fao, groups, left_on = "Item Code",right_on="Item Code",how='inner')

print("\tFiltering columns")
data_fao_colnames = ["ID","Area Code","Area","Item Code","Item_x","Element Code","Element","Unit","Description","Food group"]
data_fao_colnames.extend(years_fao_colnames)
data_fao_groups = data_fao_groups[data_fao_colnames]
data_fao_groups_colnames = ["id","area_code","area","item_code","item","element_code","element","unit","description","food_group"]
data_fao_groups_colnames.extend(years_fao_colnames)
data_fao_groups.columns = data_fao_merged_colnames

print("\tFiltering element " + str(fao_element))
data_fao_groups = data_fao_groups[data_fao_groups.element_code == fao_element]

print("\tSummarizing food groups")
food_group = data_fao_groups.food_group.unique()
data_fao_groups_colnames_fg = ["area","food_group"]
data_fao_groups = data_fao_groups.groupby(data_fao_groups_colnames_fg)[years_fao_colnames].sum().reset_index()

print("\tSaving")
data_fao_groups.to_excel(path_outputs +'data_fao_groups.xlsx', engine='openpyxl', index = False)

print("\tSuccessful")

Merging fao data with groups
	Loading fao data
	Merging fao data with groups
	Filtering columns
	Filtering element 664
	Summarizing food groups
	Saving
	Successful


In [112]:
print("Merging fao data with diets")

print("\tMerging fao data with diet")
data_fao_diet = pd.merge(data_fao_groups, diets, left_on = "food_group",right_on="food_group",how='inner')

print("\tSaving")
data_fao_diet.to_excel(path_outputs +'data_fao_diet.xlsx', engine='openpyxl', index = False)

print("\tSuccessful")

Merging fao data with diets
	Merging fao data with diet
	Saving
	Successful


In [137]:
print("Calculating categories")

print("\tCreating categories fields")
data_fao_cat = data_fao_diet.copy()
data_fao_cat["category"] = 0

print("\tGetting food groups")
food_group = data_fao_cat.food_group.unique()


quintals = pd.DataFrame()
diet = "d" + str(fao_element)
for fg in food_group:
    print("\t\tCalculating food group: " + fg)
    fg_fao = data_fao_cat[data_fao_cat.food_group == fg]
    
    for y in years_fao_colnames:
        print("\t\t\tCalculating year: " + y)
        
        # Food groups above
        fg_fao_a = fg_fao[fg_fao.type == "A"] 
        if(fg_fao_a.shape[0]>0):
            # Splitting in two datasets: above or below of diet
            fg_fao_a_above = fg_fao_a[fg_fao_a[y] >= fg_fao_a[diet]]
            fg_fao_a_below = fg_fao_a[fg_fao_a[y] < fg_fao_a[diet]]
            
            # Calculating quintals for below data
            q = fg_fao_a_below[y].quantile([0.25, 0.5,0.75, 1])
            q["year"] = y.replace("Y","")            
            q["food_group"] = fg
            q["type"] = "A"
            quintals = quintals.append(q, ignore_index=True)
            
            # Setting category A
            data_fao_cat["category"].iloc[fg_fao_a_above.index] = 5
            fg_fao_a_below["category"] = fg_fao_a_below[y].apply(lambda x: 1 if x<q[0] else (2 if x<q[1] else (3 if x<q[2] else 4)))
            data_fao_cat["category"].iloc[fg_fao_a_below.index] = fg_fao_a_below["category"]
        
        # Food groups below
        fg_fao_b = fg_fao[fg_fao.type == "B"] 
        if(fg_fao_b.shape[0]>0):
            # Splitting in two datasets: above or below of diet
            fg_fao_b_below = fg_fao_b[fg_fao_b[y] <= fg_fao_b[diet]]
            fg_fao_b_above = fg_fao_b[fg_fao_b[y] > fg_fao_b[diet]]
            
            # Calculating quintals for below data
            q = fg_fao_b_above[y].quantile([0.25, 0.5,0.75, 1])
            q["year"] = y.replace("Y","")            
            q["food_group"] = fg
            q["type"] = "B"
            quintals = quintals.append(q, ignore_index=True)
            
            # Setting category B
            data_fao_cat["category"].iloc[fg_fao_b_below.index] = 5
            fg_fao_b_above["category"] = fg_fao_b_above[y].apply(lambda x: 1 if x>q[3] else (2 if x>q[2] else (3 if x>q[1] else 4)))
            data_fao_cat["category"].iloc[fg_fao_b_above.index] = fg_fao_b_above["category"]

print("\tSaving")
quintals.to_excel(path_outputs +'quintals.xlsx', engine='openpyxl', index = False)  
data_fao_cat.to_excel(path_outputs +'data_fao_cat.xlsx', engine='openpyxl', index = False)

print("\tSuccessful")

Calculating categories
	Creating categories fields
	Getting food groups
		Calculating food group: Alcohol
			Calculating year: Y2013
		Calculating food group: Dairy

C:\Users\hsotelo\anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\hsotelo\anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



			Calculating year: Y2013
		Calculating food group: Fats & Oils
			Calculating year: Y2013
		Calculating food group: Fruits
			Calculating year: Y2013
		Calculating food group: Grains (Starchy staples)
			Calculating year: Y2013
		Calculating food group: Legumes, Nuts & Seeds
			Calculating year: Y2013
		Calculating food group: Meats
			Calculating year: Y2013
		Calculating food group: Miscellaneous
			Calculating year: Y2013
		Calculating food group: Spices
			Calculating year: Y2013
		Calculating food group: Sugars
			Calculating year: Y2013
		Calculating food group: Vegetables
			Calculating year: Y2013
	Saving
	Successful


In [143]:
print("Creating matrix for countries analysis")

print("\tPivoting table by country")
data_countries = pd.pivot_table(data_fao_cat, values='category', index=['area'], columns=['type'], aggfunc=np.sum)
print(data_countries)
#data_countries = data_countries.to_records()
#data_countries.columns = [hdr.replace("('pills', ", "strength.").replace(")", "") \
#                     for hdr in flattened.columns]

Creating matrix for countries analysis
	Pivoting table by country
type                                        A   B
area                                             
Afghanistan                                11  26
Africa                                     16  26
Albania                                    22  25
Algeria                                    16  26
Americas                                   16  21
Angola                                     15  26
Antigua and Barbuda                        10  25
Argentina                                  12  22
Armenia                                    14  23
Asia                                       16  25
Australia                                  13  19
Australia & New Zealand                    13  17
Austria                                    15  18
Azerbaijan                                 12  26
Bahamas                                    13  19
Bangladesh                                 10  28
Barbados                          